# Relevance Score notebook 

### Functionality:
* 1. Reads the news extracted from Google
* 2. Extract the title of the news 
* 3. Tokenizes the titles to calulcate the frequency (tf, idf) of the target word
* 4. Creates a score based on the frequency calculated 


In [1]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import pickle
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
#If the code is run in Google Collab
#from google.colab import drive
#drive.mount('/content/drive')
#Load the data
#import pandas as pd
#df = pd.read_csv('/content/drive/MyDrive/01_GDELT_SUPER.csv')

Mounted at /content/drive


In [39]:
df = pd.read_csv('01_GDELT_SUPER.csv')

FileNotFoundError: ignored

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18439 entries, 0 to 18438
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     18439 non-null  int64  
 1   url            18439 non-null  object 
 2   url_mobile     10364 non-null  object 
 3   title          18429 non-null  object 
 4   socialimage    14135 non-null  object 
 5   domain         18439 non-null  object 
 6   language       18439 non-null  object 
 7   sourcecountry  18106 non-null  object 
 8   date           18439 non-null  object 
 9   Subject        0 non-null      float64
dtypes: float64(1), int64(1), object(8)
memory usage: 1.4+ MB


In [9]:
df

,Unnamed: 0,url,url_mobile,title,socialimage,domain,language,sourcecountry,date,Subject
0,0,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01 00:00:00,NaN
1,1,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01 00:00:00,NaN
2,2,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01 00:00:00,NaN
3,3,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01 00:00:00,NaN
4,4,https://www.finanznachrichten.de/nachrichten-2...,NaN,"Crypto : It Complicated , But Also Very Simple",https://fns1.de/g/fb.png,finanznachrichten.de,English,Germany,2022-05-01 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...
18434,18434,https://www.etfdailynews.com/2022/04/06/polyma...,https://www.etfdailynews.com/2022/04/06/polyma...,Polymath ( POLY ) Tops 1 - Day Volume of $23 ....,NaN,etfdailynews.com,English,United States,2022-04-06 00:00:00,NaN
18435,18435,https://www.etfdailynews.com/2022/04/06/oxen-o...,https://www.etfdailynews.com/2022/04/06/oxen-o...,Oxen ( OXEN ) Trading 8 . 5 % Lower Over Last ...,NaN,etfdailynews.com,English,United States,2022-04-06 00:00:00,NaN
18436,18436,https://www.morningstar.com/articles/1087440/u...,NaN,Understanding Trends in the Cryptocurrency Mar...,https://im.morningstar.com/content/CMSImages/2...,morningstar.com,English,China,2022-04-06 00:00:00,NaN
18437,18437,https://www.etfdailynews.com/2022/04/06/nervos...,https://www.etfdailynews.com/2022/04/06/nervos...,Nervos Network ( CKB ) Trading Down 9 . 7 % Ov...,NaN,etfdailynews.com,English,United States,2022-04-06 00:00:00,NaN


In [10]:
df['date'] = pd.to_datetime(df['date'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18439 entries, 0 to 18438
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     18439 non-null  int64         
 1   url            18439 non-null  object        
 2   url_mobile     10364 non-null  object        
 3   title          18429 non-null  object        
 4   socialimage    14135 non-null  object        
 5   domain         18439 non-null  object        
 6   language       18439 non-null  object        
 7   sourcecountry  18106 non-null  object        
 8   date           18439 non-null  datetime64[ns]
 9   Subject        0 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 1.4+ MB


In [12]:
news = df

In [13]:
news.head()

,Unnamed: 0,url,url_mobile,title,socialimage,domain,language,sourcecountry,date,Subject
0,0,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01,NaN
1,1,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01,NaN
2,2,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01,NaN
3,3,https://andrecronje.medium.com/the-rise-and-fa...,NaN,The rise and fall of crypto culture,NaN,andrecronje.medium.com,English,United States,2022-05-01,NaN
4,4,https://www.finanznachrichten.de/nachrichten-2...,NaN,"Crypto : It Complicated , But Also Very Simple",https://fns1.de/g/fb.png,finanznachrichten.de,English,Germany,2022-05-01,NaN


In [14]:
news.to_csv('news_relevance.csv')

In [15]:
df_news = news[['title']]

In [16]:
df_news.title=[entry.lower() for entry in df_news.title.astype(str)]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
SW=['title:','organization:','thanks','thank','re:']

In [18]:
for sw in SW:
    df_news.title=df_news.title.replace(to_replace=sw,value='',regex=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [19]:
df_news.content =df_news.title.replace(to_replace='from:(.*\n)',value='',regex=True) #remove from to email 
df_news.content =df_news.title.replace(to_replace='lines:(.*\n)',value='',regex=True)
df_news.content =df_news.title.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
df_news.content =df_news.title.replace(to_replace='-',value=' ',regex=True)
df_news.content =df_news.title.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_news.content =df_news.title.replace(to_replace='  ',value='',regex=True)                #remove double white space
df_news.content =df_news.title.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [20]:
df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.title]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
    return file_clean_k


In [22]:
df_news.shape 

(18439, 2)

In [23]:
## This Function took around 13 hours for word Lemmatized and remove the Stop words & single character of word of each 11314 rows.
df_clean = wordLemmatizer(df_news['Word tokenize']) 
df_clean

,Keyword_final
0,"['rise', 'fall', 'crypto', 'culture']"
1,"['rise', 'fall', 'crypto', 'culture']"
2,"['rise', 'fall', 'crypto', 'culture']"
3,"['rise', 'fall', 'crypto', 'culture']"
4,"['crypto', 'complicate', 'also', 'simple']"
...,...
18434,"['polymath', 'poly', 'top', 'day', 'volume', '..."
18435,"['ox', 'ox', 'trading', 'low', 'last', 'week']"
18436,"['understand', 'trend', 'cryptocurrency', 'mar..."
18437,"['nervos', 'network', 'ckb', 'trade', 'last', ..."


In [24]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

In [25]:
df_news.insert(loc=2, column='Clean_Keyword', value=df_clean['Keyword_final'])
df_news.insert(loc=2, column='date', value=df['date'])
df_news.insert(loc=2, column='url', value=df['url'])

In [26]:
df_news_save= df_news
df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword' ],axis=1)
df_news_save

,title,url,date
0,the rise and fall of crypto culture,https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01
1,the rise and fall of crypto culture,https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01
2,the rise and fall of crypto culture,https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01
3,the rise and fall of crypto culture,https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01
4,"crypto : it complicated , but also very simple",https://www.finanznachrichten.de/nachrichten-2...,2022-05-01
...,...,...,...
18434,polymath ( poly ) tops 1 - day volume of $23 ....,https://www.etfdailynews.com/2022/04/06/polyma...,2022-04-06
18435,oxen ( oxen ) trading 8 . 5 % lower over last ...,https://www.etfdailynews.com/2022/04/06/oxen-o...,2022-04-06
18436,understanding trends in the cryptocurrency mar...,https://www.morningstar.com/articles/1087440/u...,2022-04-06
18437,nervos network ( ckb ) trading down 9 . 7 % ov...,https://www.etfdailynews.com/2022/04/06/nervos...,2022-04-06


In [28]:
df_news_save.to_csv("df_news_index.csv", index=False, header=True)

In [27]:
df_news['Clean_Keyword'] = df_news['Clean_Keyword'].astype(str)
df_news['Clean_Keyword']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0                            rise,fall,crypto,culture
1                            rise,fall,crypto,culture
2                            rise,fall,crypto,culture
3                            rise,fall,crypto,culture
4                       crypto,complicate,also,simple
                             ...                     
18434            polymath,poly,top,day,volume,million
18435                     ox,ox,trading,low,last,week
18436    understand,trend,cryptocurrency,market,chart
18437              nervos,network,ckb,trade,last,week
18438                 bitcoin,cash,price,last,day,bch
Name: Clean_Keyword, Length: 18439, dtype: object

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Create Vocabulary
vocabulary = set()

for doc in df_news.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

# Fit the TfIdf model
tfidf.fit(df_news.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(df_news.Clean_Keyword)

In [30]:
vocabulary[0:4]

['ampyx', 'inflación', 'breakup', 'suddenly']

In [31]:
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))
    
    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

In [32]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [33]:
def cosine_similarity_T(k, query):
    #print("Cosine Similarity")
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    #print("\nQuery:", query)
    #print("")
    #print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector_T(q_df['q_clean'])
    
    for d in tfidf_tran.A:
        
        d_cosines.append(cosine_sim(query_vector, d))
                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    #print(out)
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'title'] = df_news['title'][index]
        a.loc[i,'date'] = df_news['date'][index]
        a.loc[i,'url'] = df_news['url'][index]                  
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Relevance Score'] = simScore
    return a

In [34]:
df_news

,title,Word tokenize,url,date,Clean_Keyword
0,the rise and fall of crypto culture,"[the, rise, and, fall, of, crypto, culture]",https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01,"rise,fall,crypto,culture"
1,the rise and fall of crypto culture,"[the, rise, and, fall, of, crypto, culture]",https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01,"rise,fall,crypto,culture"
2,the rise and fall of crypto culture,"[the, rise, and, fall, of, crypto, culture]",https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01,"rise,fall,crypto,culture"
3,the rise and fall of crypto culture,"[the, rise, and, fall, of, crypto, culture]",https://andrecronje.medium.com/the-rise-and-fa...,2022-05-01,"rise,fall,crypto,culture"
4,"crypto : it complicated , but also very simple","[crypto, :, it, complicated, ,, but, also, ver...",https://www.finanznachrichten.de/nachrichten-2...,2022-05-01,"crypto,complicate,also,simple"
...,...,...,...,...,...
18434,polymath ( poly ) tops 1 - day volume of $23 ....,"[polymath, (, poly, ), tops, 1, -, day, volume...",https://www.etfdailynews.com/2022/04/06/polyma...,2022-04-06,"polymath,poly,top,day,volume,million"
18435,oxen ( oxen ) trading 8 . 5 % lower over last ...,"[oxen, (, oxen, ), trading, 8, ., 5, %, lower,...",https://www.etfdailynews.com/2022/04/06/oxen-o...,2022-04-06,"ox,ox,trading,low,last,week"
18436,understanding trends in the cryptocurrency mar...,"[understanding, trends, in, the, cryptocurrenc...",https://www.morningstar.com/articles/1087440/u...,2022-04-06,"understand,trend,cryptocurrency,market,chart"
18437,nervos network ( ckb ) trading down 9 . 7 % ov...,"[nervos, network, (, ckb, ), trading, down, 9,...",https://www.etfdailynews.com/2022/04/06/nervos...,2022-04-06,"nervos,network,ckb,trade,last,week"


In [35]:
coins = ['bitcoin', 'ethereum','litecoin', 'bitcoin cash', 'bitcoin sv', 'polkadot', 'chainlink', 'binance coin',
 've chain', 'neo', 'tezos', 'tether', 'usd coin', 'monero', 'dash', 'zcash', 'ripple', 'cardano', 'stellar']

In [36]:
final_dataframe = pd.DataFrame()
for i in coins:
    #final_loop = pd.DataFrame()
    final_loop = cosine_similarity_T(20,i)
    final_loop['coin'] = str(i) 
    final_dataframe =final_dataframe.append(final_loop)

In [37]:
RelevanceScore = final_dataframe.reset_index()
RelevanceScore = RelevanceScore.drop(['level_0', 'index'], axis = 1)
RelevanceScore

,title,date,url,Relevance Score,coin
0,now you can own bitcoin in 401 ( k ) s . shoul...,2022-04-27,https://www.kiplinger.com/investing/cryptocurr...,1.000000,bitcoin
1,bitcoin : is bitcoin mining is legal ?,2022-04-10,https://www.marca.com/en/lifestyle/2022/04/10/...,0.631420,bitcoin
2,how to buy bitcoin in 2022,2022-04-08,https://baltimorepostexaminer.com/how-to-buy-b...,0.547040,bitcoin
3,"bitcoin falls through $40 , 000",2022-04-22,https://www.forexlive.com/Cryptocurrency/bitco...,0.513773,bitcoin
4,"bitcoin falls 5 . 3 % to $39 , 881",2022-04-11,https://www.933thedrive.com/2022/04/11/bitcoin...,0.513773,bitcoin
...,...,...,...,...,...
375,"stellar demand , record low dealers in blockbu...",2022-04-26,https://www.zerohedge.com/markets/stellar-dema...,0.356885,stellar
376,the chrysalis brew project receives 2022 stell...,2022-04-20,https://www.digitaljournal.com/pr/the-chrysali...,0.350302,stellar
377,capitec leads the banking pack with stellar an...,2022-04-12,https://www.dailymaverick.co.za/article/2022-0...,0.342747,stellar
378,newly - discovered stellar explosion the micr...,2022-04-23,https://www.natureworldnews.com/articles/50503...,0.341086,stellar


In [38]:
RelevanceScore.to_csv('RelevanceScore.csv')